from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import string
import os
import sys
import shutil
import openai
#os.chdir('/content/drive/MyDrive/Sanjiv/IMP-OIC-Windowing')
from utils.extractframes import FrameExtractor
import graphene
directory_path = 'STAR_eval/Charades'
from STAR_eval.gpt_ask import run_gpt
main_ds = pd.read_json('STAR_eval/STAR_val.json')
main = main_ds.loc[:, ['question_id','question','video_id','start','end','answer', 'choices']]


In [ ]:
CLASSES = ['airplane', 'animal', 'arm', 'bag', 'banana', 'basket', 'beach', 'bear', 'bed', 'bench', 'bike',
                'bird', 'board', 'boat', 'book', 'boot', 'bottle', 'bowl', 'box', 'boy', 'branch', 'building',
                'bus', 'cabinet', 'cap', 'car', 'cat', 'chair', 'child', 'clock', 'coat', 'counter', 'cow', 'cup',
                'curtain', 'desk', 'dog', 'door', 'drawer', 'ear', 'elephant', 'engine', 'eye', 'face', 'fence',
                'finger', 'flag', 'flower', 'food', 'fork', 'fruit', 'giraffe', 'girl', 'glass', 'glove', 'guy',
                'hair', 'hand', 'handle', 'hat', 'head', 'helmet', 'hill', 'horse', 'house', 'jacket', 'jean',
                'kid', 'kite', 'lady', 'lamp', 'laptop', 'leaf', 'leg', 'letter', 'light', 'logo', 'man', 'men',
                'motorcycle', 'mountain', 'mouth', 'neck', 'nose', 'number', 'orange', 'pant', 'paper', 'paw',
                'people', 'person', 'phone', 'pillow', 'pizza', 'plane', 'plant', 'plate', 'player', 'pole', 'post',
                'pot', 'racket', 'railing', 'rock', 'roof', 'room', 'screen', 'seat', 'sheep', 'shelf', 'shirt',
                'shoe', 'short', 'sidewalk', 'sign', 'sink', 'skateboard', 'ski', 'skier', 'sneaker', 'snow',
                'sock', 'stand', 'street', 'surfboard', 'table', 'tail', 'tie', 'tile', 'tire', 'toilet', 'towel',
                'tower', 'track', 'train', 'tree', 'truck', 'trunk', 'umbrella', 'vase', 'vegetable', 'vehicle',
                'wave', 'wheel', 'window', 'windshield', 'wing', 'wire', 'woman', 'zebra']

REL_CLASSES = ['above', 'across', 'against', 'along', 'and', 'at', 'attached to', 'behind',
                'belonging to', 'between', 'carrying', 'covered in', 'covering', 'eating', 'flying in', 'for',
                'from', 'growing on', 'hanging from', 'has', 'holding', 'in', 'in front of', 'laying on',
                'looking at', 'lying on', 'made of', 'mounted on', 'near', 'of', 'on', 'on back of', 'over',
                'painted on', 'parked on', 'part of', 'playing', 'riding', 'says', 'sitting on', 'standing on',
                'to', 'under', 'using', 'walking in', 'walking on', 'watching', 'wearing', 'wears', 'with']

words = set(CLASSES+REL_CLASSES)

In [ ]:
import string
index_list = []
for q in main.loc[:,'question_id'].values:
    questions = main.query("question_id=='"+q+"'")["question"]
    answer = main.query("question_id=='"+q+"'")["answer"].values[0].lower().strip('the').translate(str.maketrans('', '', string.punctuation))
    count = 0
    question = ''
    for que in questions:
        question = que.lower().translate(str.maketrans('', '', string.punctuation))

    for i in list(answer.strip().split(' ')):
      if i not in  ['a', 'is','by', 'was', 'which', 'man', 'did', 'do', 'woman', 'boy', 'girl', 'person','people','scene','frame','in','to','the', 'of', 'on','with', 'from', 'at', 'and']:
          if i in words:
            index_list.append(main[main['question_id']==q].index[0])

print(index_list)
#QA.drop(index_list, inplace = True)
main = main[main.index.isin(index_list)]
main.info()


In [ ]:
import ffmpeg

def segment_video(input_path, output_path, start, end):
    input_file = ffmpeg.input(directory_path+'/'+input_path+'.mp4')
    duration = int(end-start)
    if duration < 10:
      output_file = ffmpeg.output(input_file.trim(start=start, duration = duration).filter('setpts','PTS-STARTPTS'), directory_path+'/trim/'+output_path+'.mp4')
      ffmpeg.run(output_file)
      return True
    else:
      return False

In [ ]:


def run_oic(question_id, video_id, question, choices, start,end):
    TEMP_DIR = "temp"
    CAM_PATH = "cam"
    OUT_DIR = "out"

    for video in os.scandir(directory_path):
        if video.is_file():
            if os.path.splitext(video.name)[0] == video_id:
                #print('processing {}'.format(video_id))
                #print(video_id, question, choices, start, end)
                video_p = segment_video(os.path.splitext(video.name)[0],question_id+os.path.splitext(video.name)[0],start,end)
                if video_p:
                  ex = FrameExtractor(directory_path+'/trim/'+question_id+video.name, 10, 5)
                  ex.main()
                  g = graphene.Graphene(0.3, 0.6)

                  if not os.path.isdir(OUT_DIR):
                      os.mkdir(OUT_DIR)
                  text = question_id+os.path.splitext(video.name)[0]+'graph2text.txt'
                  img_path = directory_path+'/trim/'+question_id+os.path.splitext(video.name)[0]+'-opencv'


                  g.classify_images_window(img_path, 5)
                  graph_path = img_path + "/img/JSON"
                  g.generate_temporal_graph_frames(graph_path, img_path + "/img")


                  g.tg.to_text(os.path.join(OUT_DIR, text))
      
                  if os.path.isfile(os.path.join(OUT_DIR,text)):
                      with open(os.path.join(OUT_DIR,text)) as f:
                          context = "".join(map(str,f.readlines()))
                          return str(context)
                  else:
                    continue

In [ ]:
import shutil
shutil.rmtree('temp')

In [ ]:
import contextlib
import time
import warnings
import cProfile

q_ids = main['question_id'].unique()
print(len(q_ids))
for q in q_ids:
  #if 'Seq' in q:
    que = main.query("question_id == '"+q+"'")
    video_id = que['video_id'].values[0]
    question = que['question'].values[0]
    answer = que['answer'].values[0].lower().strip('the ').translate(str.maketrans('', '', string.punctuation))
    start = que['start'].values[0]
    end = que['end'].values[0]
    choices = dict()
    choice_string = ''
    options = main.query("question_id == '"+q+"'")["choices"].values[0]
    print(options)
    for choice in options:
          choices.update({choice['choice_id']:choice['choice'].lower().strip('the').translate(str.maketrans('', '', string.punctuation)).strip()})
          choice_string += ' ('+str(choice['choice_id']+1)+')'+str(choice['choice'].lower().strip('the').translate(str.maketrans('', '', string.punctuation)))

    if not os.path.isdir(directory_path+'/trim/'+q+video_id+'-opencv'):
      cProfile.run('run_oic(q, video_id, question, choices, start, end)')
      prompt = run_oic(q, video_id, question, choices, start, end)
      if prompt is not None:
        main.loc[main['question_id'] == q, 'OIC_context'] = prompt
        formated_question = question+' Guess the most likely answer among these four options: '+choice_string+' Respond only with a single number between 1 and 4. Do not produce any other output.'
        response = run_gpt(prompt, formated_question)
        main.loc[main['question_id'] == q, 'OIC_answer'] = str(response)
        main.loc[main['question_id'] == q, 'OIC_question'] = formated_question
        OIC_answer = response
        print(choices)
        print (int(OIC_answer))
        print(choices[int(OIC_answer)-1])
        print(answer)
        if choices[int(OIC_answer)-1] == answer:
          main.loc[main['question_id'] == q, 'Match'] = 'Correct'
          print('correct')
        else:
          print('wrong')
          main.loc[main['question_id'] == q, 'Match'] = 'Wrong'
        print('------------------------------------')
        print('OIC question: {}'.format(formated_question))
        print('OIC answer: {}'.format(choices[int(OIC_answer)-1]))
        main.to_csv('STAR_eval/datatables4/question_answers.csv')
          
main.head()
main.name = 'all data'